# 数据处理

为构建我们的本地知识库，我们需要对以多种类型存储的本地文档进行处理，读取本地文档并通过前文描述的 Embedding 方法将本地文档的内容转化为词向量来构建向量数据库。在本节中，我们以一些实际示例入手，来讲解如何对本地文档进行处理。
## 一、源文档选取
我们选用 Datawhale 一些经典开源课程作为示例，具体包括：
* [《机器学习公式详解》PDF版本](https://github.com/datawhalechina/pumpkin-book/releases)
* [《面向开发者的LLM入门教程、第一部分Prompt Engineering》md版本](https://github.com/datawhalechina/llm-cookbook)  
我们将知识库源数据放置在../../data_base/knowledge_db 目录下。
## 二、数据读取
**PDF文档为例**  
我们可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [6]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../../data_base/knowledge_db/ZXVMAX-S/ZXVMAX-S（V6.20.80.02）快速入门.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

In [7]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf")
# pages = loader.load_and_split()

文档加载后储存在 `pages` 变量中:
- `page` 的变量类型为 `List`
- 打印 `pages` 的长度可以看到 pdf 一共包含多少页

In [8]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 18 页


`page` 中的每一元素为一个文档，变量类型为 `langchain_core.documents.base.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [9]:
pdf_page = pdf_pages[0]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../../data_base/knowledge_db/ZXVMAX-S/ZXVMAX-S（V6.20.80.02）快速入门.pdf', 'file_path': '../../data_base/knowledge_db/ZXVMAX-S/ZXVMAX-S（V6.20.80.02）快速入门.pdf', 'page': 0, 'total_pages': 18, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'creator': 'DITA Open Toolkit', 'producer': 'Apache FOP Version 2.3', 'creationDate': "D:20220623163150+08'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
ZXVMAX-S
多维价值分析系统
快速入门
产品版本：V6.20.80.03
中兴通讯股份有限公司
地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦
邮编：518057
电话：0755-26770800
   400-8301118
      
800-8301118（座机）
技术支持网站：http://support.zte.com.cn
电子邮件：800@zte.com.cn



In [6]:
# 环境中缺少 NLTK 数据包中的 punkt 资源，这是 NLTK 库中用于分词的一个重要组件。解决这个问题的方法是按照报错信息中提到的步骤下载并安装 punkt 资源。
# import nltk
# nltk.download('punkt', quiet=True)

## 三、数据清洗
我们期望知识库的数据尽量是有序的、优质的、精简的，因此我们要删除低质量的、甚至影响理解的文本数据。  
可以看到上文中读取的pdf文件不仅将一句话按照原文的分行添加了换行符`\n`，也在原本两个符号中间插入了`\n`，我们可以使用正则表达式匹配并删除掉`\n`。

In [7]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
pdf_page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), pdf_page.page_content)
print(pdf_page.page_content)

ZXVMAX-S
多维价值分析系统
快速入门
产品版本：V6.20.80.03
中兴通讯股份有限公司
地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦
邮编：518057
电话：0755-26770800   400-8301118      800-8301118（座机）
技术支持网站：http://support.zte.com.cn
电子邮件：800@zte.com.cn



进一步分析数据，我们发现数据中还有不少的`•`和空格，我们的简单实用replace方法即可。

In [12]:
pdf_page.page_content = pdf_page.page_content.replace('•', '')
pdf_page.page_content = pdf_page.page_content.replace(' ', '')
print(pdf_page.page_content)

法律声明
本资料著作权属中兴通讯股份有限公司所有。未经著作权人书面许可，任何单位或个人不得以任何方式
摘录、复制或翻译。
侵权必究。 
和 是中兴通讯股份有限公司的注册商标。中兴通讯产品的名称和标志是中兴通讯的专有
标志或注册商标。在本手册中提及的其他产品或公司的名称可能是其各自所有者的商标或商名。在未经
中兴通讯或第三方商标或商名所有者事先书面同意的情况下，本手册不以任何方式授予阅读者任何使用
本手册上出现的任何标记的许可或权利。
本产品符合关于环境保护和人身安全方面的设计要求，产品的存放、使用和弃置应遵照产品手册、相关
合同或相关国法律、法规的要求进行。
如果本产品进行改进或技术变更，恕不另行专门通知。
当出现产品改进或者技术变更时，您可以通过中兴通讯技术支持网站http://support.zte.com.cn查询有
关信息。 
第三方嵌入式软件使用限制声明：
如果与本产品配套交付了Oracle、Sybase/SAP、Veritas、Microsoft、VMware、Redhat这些第三方嵌入
式软件，只允许作为本产品的组件，与本产品捆绑使用。当本产品被废弃时，这些第三方软件的授权许
可同时作废，不可转移。这些嵌入式软件由ZTE给最终用户提供技术支持。
修订历史
资料版本
发布日期
更新说明
R1.22022-6-20
更新菜单路径和界面图
R1.12021-05-12
更新菜单路径和界面图
R1.02020-12-30
第一次发布
资料编号∶SJ-20220623151803-001
发布日期∶2022-06-20（R1.2）



上文中读取的md文件每一段中间隔了一个换行符，我们同样可以使用replace方法去除。

## 四、文档分割

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../../figures/C3-3-example-splitter.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [14]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [16]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(pdf_page.page_content[0:1000])

['法律声明\n本资料著作权属中兴通讯股份有限公司所有。未经著作权人书面许可，任何单位或个人不得以任何方式\n摘录、复制或翻译。\n侵权必究。\xa0\n和\xa0是中兴通讯股份有限公司的注册商标。中兴通讯产品的名称和标志是中兴通讯的专有\n标志或注册商标。在本手册中提及的其他产品或公司的名称可能是其各自所有者的商标或商名。在未经\n中兴通讯或第三方商标或商名所有者事先书面同意的情况下，本手册不以任何方式授予阅读者任何使用\n本手册上出现的任何标记的许可或权利。\n本产品符合关于环境保护和人身安全方面的设计要求，产品的存放、使用和弃置应遵照产品手册、相关\n合同或相关国法律、法规的要求进行。\n如果本产品进行改进或技术变更，恕不另行专门通知。\n当出现产品改进或者技术变更时，您可以通过中兴通讯技术支持网站http://support.zte.com.cn查询有\n关信息。\xa0\n第三方嵌入式软件使用限制声明：\n如果与本产品配套交付了Oracle、Sybase/SAP、Veritas、Microsoft、VMware、Redhat这些第三方嵌入',
 '式软件，只允许作为本产品的组件，与本产品捆绑使用。当本产品被废弃时，这些第三方软件的授权许\n可同时作废，不可转移。这些嵌入式软件由ZTE给最终用户提供技术支持。\n修订历史\n资料版本\n发布日期\n更新说明\nR1.22022-6-20\n更新菜单路径和界面图\nR1.12021-05-12\n更新菜单路径和界面图\nR1.02020-12-30\n第一次发布\n资料编号∶SJ-20220623151803-001\n发布日期∶2022-06-20（R1.2）']

In [17]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：21


In [18]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：5408


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在本章，我们仅简单根据 chunk_size 对文档进行分割。对于有兴趣进一步探索的读者，欢迎阅读我们第三部分的项目示例来参考已有的项目是如何进行文档分割的。